In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import numpy as np
import cv2
from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten,Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow.compat.v1 as tf
tf.logging.set_verbosity(tf.logging.ERROR)
a = tf.zeros(1)

## Model

#### Import VGG16 with imagenet weights

In [ ]:
Dimension = (224,224)
VGG = vgg16.VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (Dimension[0], Dimension[1], 3))
# Stopping the convolutional layers
for layer in VGG.layers:
    layer.trainable = False
    

#### Adding Layers

In [ ]:
CNN = VGG.output
CNN = Conv2D(512,kernel_size = (3,3),strides = (1,1),activation = "relu",padding = "same")(CNN)
CNN = Conv2D(512,kernel_size = (3,3),strides = (1,1),activation = "relu",padding = "same")(CNN)
CNN = Flatten(name = "flatten")(CNN)
CNN = Dense(256, activation = "relu")(CNN)
CNN = Dense(256, activation = "relu")(CNN)
CNN = Dense(4, activation = "softmax")(CNN)

model = Model(inputs = VGG.input, outputs = CNN, name = "CustomModel")
model.summary()

### Dataset

In [ ]:
train_data_dir = '../input/kermany2018/OCT2017 /train'
validation_data_dir = '../input/kermany2018/OCT2017 /test'
test_data_dir = '../input/kermany2018/OCT2017 /val'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)


batch_size=32
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(Dimension[0], Dimension[1]),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(Dimension[0], Dimension[1]),
        batch_size=batch_size,
        class_mode='categorical')

testing_generator = validation_datagen.flow_from_directory(
        test_data_dir,
        target_size=(Dimension[0], Dimension[1]),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
checkpoint = ModelCheckpoint("retinal.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 2,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]

### Training

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(learning_rate = 0.0001),
              metrics = ['accuracy'])
history = model.fit_generator(
    train_generator,
    epochs = 20,
    callbacks = callbacks,
    validation_data = validation_generator)

### Loading Models

In [ ]:
Model = '../input/models/retinal_vgg1_00001_16.h5'
model = load_model(Model)

### Accuracy

In [ ]:
scores = model.evaluate_generator(validation_generator, verbose=1)
print('\nTest result: %.3f loss: %.3f' %(scores[1]*100,scores[0]))

In [ ]:
# Loss Curves
plt.figure(figsize=[10,8])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.grid()
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.grid()
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16);


In [ ]:
y_pred = model.predict(validation_generator)
y_pred_label = np.argmax(y_pred, axis=1)
y_pred=[ np.argmax(Y_pred[i]) for i in range(validation_generator.samples)]
y_pred

In [ ]:
class_labels = validation_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
y_pred = model.predict(validation_generator, 968 // batch_size+1)
y_pred_label = np.argmax(y_pred, axis=1)
y_pred = [np.where(predictions>0.5, 1, 0) for predictions in Y_pred]
validation_generator.classes.shape,y_pred_label.shape
#Confusion Matrix and Classification Report
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred_label))
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred_label, target_names=classes))

### Hyperparameter Tuning

In [ ]:
Accuracy = [94.215,96.901,97.004]
Losses = [0.168,0.125,0.094]
Models = ["0.001 (16)","0.0001 (16)", "0.0001 (32)"]

plt.scatter(Models,Accuracy)
plt.xlabel('Hyperparameters: LR (Batch Size) ',fontsize=10)
plt.ylabel('Accuracy',fontsize=10)
plt.title('Hyperparameter Tuning',fontsize=16);
plt.show()

In [ ]:
plt.scatter(Models,Losses)
plt.xlabel('Hyperparameters: LR (Batch Size) ',fontsize=10)
plt.ylabel('Loss',fontsize=10)
plt.title('Hyperparameter Tuning',fontsize=16);
plt.show()

In [ ]:
plt.scatter(["VGG16","Custom"],[93.49,96.901])
plt.xlabel('Models',fontsize=10)
plt.ylabel('Accuracy',fontsize=10)
plt.title('VGG vs Custom',fontsize=16);
plt.show()